<a href="https://colab.research.google.com/github/nyanta012/demo/blob/main/rinna_4b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install transformers
!pip install sentencepiece
!pip install accelerate
!pip install bitsandbytes

# 4b-instruction-stf

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("rinna/bilingual-gpt-neox-4b-instruction-sft", use_fast=False)
model = AutoModelForCausalLM.from_pretrained("rinna/bilingual-gpt-neox-4b-instruction-sft")

You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


In [3]:
def response_generator(input_user: str) -> None:
    global model
    prompt = [
        {
            "speaker": "ユーザー",
            "text": input_user
        },
    ]
    prompt = [
        f"{uttr['speaker']}: {uttr['text']}"
        for uttr in prompt
    ]
    prompt = "<NL>".join(prompt)
    prompt = (
        prompt
        + "<NL>"
        + "システム: "
    )

    if torch.cuda.is_available():
        model = model.to("cuda")

    token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")

    with torch.no_grad():
      output_ids = model.generate(
          token_ids.to(model.device),
          max_new_tokens=512,
          do_sample=True,
          temperature=0.1,
          repetition_penalty=1.05,
          top_p=0.85,
          pad_token_id=tokenizer.pad_token_id,
          bos_token_id=tokenizer.bos_token_id,
          eos_token_id=tokenizer.eos_token_id
      )

    output = tokenizer.decode(output_ids.tolist()[0][token_ids.size(1):])
    output = output.replace("<NL>", "\n")
    print(output)

In [10]:
input_sentence = """
データサイエンティストに必要なことを5個教えてください
"""
response_generator(input_sentence)

データサイエンティストに必要なことは、以下のとおりです:

- データ分析のスキル
- データ分析のツールとテクニック
- データ分析の実践
- データ分析の倫理的考慮事項
- データ分析の実践における倫理的考慮事項</s>


## 文脈から答えさせる

In [11]:
input_sentence = """
質問 : レアゾンスピーチについて教えてください
"""
response_generator(input_sentence)

レアゾンスピーチとは、1960年代にアメリカで放送されたテレビ番組です。このショーは、アメリカのテレビネットワークであるNBCによって放送されました。この番組は、アメリカのテレビネットワークが制作した最初の番組であり、また、最初のアメリカのテレビネットワークの1つでもありました。</s>


In [12]:
input_sentence = """
文脈 ： レアゾンスピーチとは、株式会社レアゾン・ホールディングスが開発した音声認識に関するモデルやデータの総称です。
今回は音声認識モデルについて紹介します。
音声認識モデルとは、音声をテキストに変換するAIのことで、文字起こしや字幕作成などに使われます。

質問 ： レアゾンスピーチについて教えてください
"""
response_generator(input_sentence)

はい、レアゾンスピーチとは、株式会社レアゾン・ホールディングスが開発した音声認識に関するモデルやデータの総称です。</s>


In [13]:
input_sentence = """
文脈 ： レアゾンスピーチとは、株式会社レアゾン・ホールディングスが開発した音声認識に関するモデルやデータの総称です。
今回は音声認識モデルについて紹介します。
音声認識モデルとは、音声をテキストに変換するAIのことで、文字起こしや字幕作成などに使われます。

質問 ： 音声認識モデルは何に使われますか？
"""
response_generator(input_sentence)

文字起こしや字幕作成</s>


## 要約タスク

In [14]:
input_sentence = """
文章 : ReazonSpeechとは、株式会社レアゾン・ホールディングスが開発した音声認識に関するモデルやデータの総称です。
今回は音声認識モデルについて紹介します。
音声認識モデルとは、音声をテキストに変換するAIのことで、文字起こしや字幕作成などに使われます。
ReazonSpeechの音声認識モデルは、ワンセグ放送の録画データから取得した19,000時間のデータを学習させたモデルで、日本語に特化したモデルと言えます。
このモデルは、商用利用可能で無償で提供されており、Googleコラボなどで簡単に文字起こしを試すことができます。

指示 : 上記の文章を要約してください

"""
response_generator(input_sentence)

ReazonSpeechは、株式会社レアゾン・ホールディングスが開発した音声認識に関するモデルやデータの総称です。</s>


In [21]:
input_sentence = """
ReazonSpeechを使って文字起こしをやってみた感想は、とても高精度で驚きました。
ReazonSpeechは、日本語の発音や文法に合わせて学習されたモデルなので、日本語で文字起こしをしたい人にとっては非常に良いモデルだと思います。
また、パラメータの数が少なくても高精度なので、計算コストが低くて済むというメリットもあります。
しかし、一方で「レアゾン」という単語が「エアゾン」と誤判定されたり、話者が早口だったりすると区切りがうまくできなかったりすることがありました。
また、長い動画に対してどれくらい精度が保たれるかや、Whisperと比較してどれくらい精度が高いかなどは、まだ自分で検証できていない部分があります。
何れにせよ日本企業が自前でAIモデルを開発して、公開するというのは珍しいので、今後も応援していきたいですね。

指示 : 上記の文章を要約してください
"""
response_generator(input_sentence)

ReazonSpeechを使用して文字起こしをやってみました。</s>


## 翻訳タスク

In [22]:
input_sentence = """
I love cats

上記を日本語にしてください
"""

response_generator(input_sentence)

私は猫が大好きです。</s>


In [23]:
input_sentence = """
In this work, we develop and release Llama 2, a collection of pretrained and fine-tuned
large language models (LLMs) ranging in scale from 7 billion to 70 billion parameters.
Our fine-tuned LLMs, called Llama 2-Chat, are optimized for dialogue use cases. Our
models outperform open-source chat models on most benchmarks we tested, and based on
our human evaluations for helpfulness and safety, may be a suitable substitute for closedsource models. We provide a detailed description of our approach to fine-tuning and safety
improvements of Llama 2-Chat in order to enable the community to build on our work and
contribute to the responsible development of LLMs.

上記を日本語にしてください
"""

response_generator(input_sentence)

このプロジェクトは、大規模な言語学習モデル(LLMs)の大規模化と自動化を通じて、7 Billionから70 Billionまでの能力を備えた、高度な機械学習モデル(LLMs)を開発しました。これらのLLMsは、チャットシナリオでの使用に最適化されています。私たちの高度な機械学習モデルは、ユーザーのフィードバックに基づいて、最も役立つツールとして評価されています。私たちは、ユーザーのフィードバックに基づいて、高度な機械学習モデルがどのように機能するかについて説明し、ユーザーのニーズを満たすために、LLMsの品質と安全性を向上させるための取り組みを紹介します。</s>


In [24]:
input_sentence = """
健康は私たちの生活において非常に重要です。生活習慣は健康に大きな影響を与えます。バ
ランスの取れた食事を心掛け、野菜や果物、たんぱく質、繊維を摂取しましょう。適度な運
動を行うことも健康に良い影響を与えます。毎日の運動の習慣を身につけましょう。十分な
睡眠を確保することも重要です。ストレスの管理も健康に欠かせません。リラックス法やマ
インドフルネス瞑想を取り入れましょう。健康と生活習慣の関係を理解し、バランスの取れ
た生活を送ることが健康への第一歩です



健康は私たちに最も必要なものだ。私たちの健康には食事と運動が大切です。
バランスの取れた食事をし、水をたくさん飲み、野菜、果物、全粒穀物、赤身のタンパク質をたっぷり摂る健康的な食生活を送る必要がある。
また、十分な睡眠と運動も必要です。健康的な体重を維持し、身体を活発に動かす必要があります。
心の健康にも気を配り、人生を前向きにとらえる必要があります。


上記を英語にしてください
"""

response_generator(input_sentence)

Health is what we need most. Diet and exercise are important for our health. We need to eat a balanced diet, drink plenty of water and eat a healthy diet full of vegetables, fruits, whole grains and lean protein. We also need to get enough sleep and exercise. We need to maintain a healthy weight and stay physically active. We need to take good care of our mental health and maintain a positive outlook on life.</s>
